In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('../Data/CleanedData.csv')

In [3]:
df.head()

,subject,transformed text
0,1,donald trump wish american happi new year leav...
1,1,hous intellig committe chairman devin nune go ...
2,1,friday reveal former milwauke sheriff david cl...
3,1,christma day donald trump announc would back w...
4,1,pope franci use annual christma day messag reb...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000)

In [5]:
x= cv.fit_transform(df['transformed text']).toarray()
y= df['subject'].values

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state = 42)

In [8]:
random_forest = RandomForestClassifier()

# Define a parameter grid to search through
param_grid = {
    'n_estimators': [100, 200],                 # Number of trees in the forest
    'max_depth': [None, 10, 20],                 # Maximum depth of the trees
    'min_samples_split': [2, 5],                # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2],                  # Minimum samples required to be at a leaf node
    'max_features': ['auto']  # Number of features to consider at each split
}

# Create a GridSearchCV object with the model and parameter grid
grid_search = GridSearchCV(random_forest, param_grid, cv=2, scoring='accuracy', verbose = 4)

# Fit the grid search to your training data
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy Score:", grid_search.best_score_)

# Use the best estimator to make predictions on the test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)



Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV 1/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.732 total time=  21.0s
[CV 2/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.728 total time=  17.5s
[CV 1/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.735 total time=  34.0s
[CV 2/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.733 total time=  35.8s
[CV 1/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.730 total time=  14.8s
[CV 2/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.729 total time=  15.1s
[CV 1/2] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200

KeyboardInterrupt: 

In [ ]:
# Evaluate the model's performance on the test data
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Test Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)
